In [2]:
import copy 

import pandas as pd
import copy

import re

import os

import operator

import pymorphy2
from pymystem3 import Mystem

import nltk

import itertools 

import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from collections import OrderedDict

import progressbar
import time

from collections import OrderedDict
from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«"
full_punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–,»«'

In [3]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.remove("бы")
russian_stopwords.remove("не")
russian_stopwords.remove("ни")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
russian_stopwords

['и',
 'в',
 'во',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'над',
 'больше',
 'тот'

In [4]:
nltk.download('punkt')
tokenizer = nltk.data.load('russian.pickle')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [5]:
text = "Ай да А.С. Пушкин! Ай да сукин сын!"
print("Before:", nltk.sent_tokenize(text))
print("After:", tokenizer.tokenize(text))

Before: ['Ай да А.С.', 'Пушкин!', 'Ай да сукин сын!']
After: ['Ай да А.С. Пушкин!', 'Ай да сукин сын!']


In [6]:
df_sence = pd.read_csv("./for calculation/mult_sence.csv")
sence_dict = pd.Series(df_sence.count_rank.values,index=df_sence.word).to_dict()

def low(word):
    return word.lower()
df_freq = pd.read_csv("./for calculation/freq_lables.csv")
df_freq['Lemma'] = df_freq['Lemma'].apply(low)
freq_pos_dict = pd.Series(df_freq.PoS.values,index=df_freq.Lemma).to_dict()
freq_dict = pd.Series(df_freq.freq_lable.values,index=df_freq.Lemma).to_dict()

df_eng = pd.read_csv("./for calculation/eng_words.csv", sep = ' ')
eng_words = set(df_eng['orth'])

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [6]:
user_vector = np.array([2.1,2.3,1.4, 0.7], np.float).reshape(1, -1)
word_vector = np.array([1,1.3, 1, 0], np.float).reshape(1, -1)
cosine_similarity(user_vector, word_vector)

array([[0.96926688]])

In [13]:
word_vector_2 = np.array([2.4,3, 2, 1], np.float).reshape(1, -1)
cosine_similarity(user_vector, word_vector_2)

array([[0.99647335]])

In [21]:
def el_wise_extract(user_vector, word_vector):
    difference = []
    #print(user_vector)
    for user_el, word_el in zip(user_vector[0], word_vector[0]):
        print(user_el,word_el )
        difference.append(user_el - word_el)
    return difference
el_wise_extract(user_vector, word_vector)

2.1 1.0
2.3 1.3
1.4 1.0
0.7 0.0


[1.1, 0.9999999999999998, 0.3999999999999999, 0.7]

In [22]:
el_wise_extract(user_vector, word_vector_2)

2.1 2.4
2.3 3.0
1.4 2.0
0.7 1.0


[-0.2999999999999998,
 -0.7000000000000002,
 -0.6000000000000001,
 -0.30000000000000004]

In [7]:
def delete_empty_newlines(file):
    end_line_punctuation = "!.?»"
    new_lines = []
    collected_line = ''
    empty_endline = False
    with open (file, "r", encoding = "utf-8") as f:
        for line in f.readlines():
            cl_line = line.strip()
            if(cl_line[-1] not in end_line_punctuation):
                collected_line += ' ' + cl_line
                empty_endline = True
            else:
                if(collected_line):
                    collected_line += ' ' + cl_line
                empty_endline = False
            
            if not (empty_endline):
                if(collected_line):
                    new_lines.append(collected_line)
                    collected_line = ''
                    
                else:
                    new_lines.append(cl_line)
                
    return new_lines
list_sentences_lines = delete_empty_newlines("text_8.txt")

In [8]:
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    clean_sentence = []
    #print(sentences)
    for sentence in sentences:
        words = sentence.split()
        clean_text= ''
        for word in words:
            clean_word = ''
            for char in word:
                if char != " " and char not in full_punctuation:
                    clean_word += char.lower()
            clean_text += clean_word + ' '
        clean_text = re.sub(' +', ' ', clean_text)
        clean_text = clean_text.strip()
        clean_sentence.append(clean_text)
    return clean_sentence   

def clean_file(file_lines_list):
    preprocessed_text = []
    for line in file_lines_list:
        t = preprocess_text(line)
        #print(t)
        preprocessed_text.extend(t)

    return preprocessed_text
sentences_list = clean_file(list_sentences_lines)

In [9]:
len(sentences_list)

25

In [22]:
word_count = 0
for sentence in sentences_list:
    word_count += len(sentence.split())
print(word_count)

133


In [9]:
with open("./adjusting texts/text_8_tokenized.parsed", "r", encoding = "utf-8") as f:
    word_count = 0
    for line in f.readlines():
        if(line[0].isdigit()):
            word_properties = line.split('\t')
            if(word_properties[3] != 'PUNCT'):
                print(word_properties[1])
                word_count += 1
    print(word_count)

В
классе
был
урок
русского
языка
Преподаватель
читал
новый
текст
Студенты
внимательно
слушали
Извините
можно
войти
Я
опоздал
сказал
Джон
Что
случилось
Джон
Вчера
вы
опоздали
сегодня
вы
опоздали
Нельзя
опаздывать
на
урок
Вечером
Джон
пришел
домой
Дома
он
долго
думал
Вчера
я
опоздал
сегодня
я
тоже
опоздал
Это
очень
плохо
Не
буду
больше
опаздывать
на
урок
Завтра
я
встану
рано
в
8
часов
быстро
сделаю
зарядку
приготовлю
завтрак
позавтракаю
повторю
домашнее
задание
Завтра
я
не
опоздаю
Утром
Джон
встал
в
8
часов
Он
сделал
зарядку
приготовил
завтрак
позавтракал
повторил
домашнее
задание
и
пошел
в
институт
В
9
часов
30
минут
Джон
пришел
в
институт
Что
случилось
молодой
человек
Куда
вы
идете
спросил
дежурный
Я
иду
на
урок
Сегодня
я
не
опоздал
ответил
Джон
Идите
домой
молодой
человек
Сегодня
нет
урока
Сегодня
воскресенье
133


In [9]:
sentences_list

['в классе был урок русского языка',
 'преподаватель читал новый текст',
 'студенты внимательно слушали',
 'извините можно войти',
 'я опоздал сказал джон',
 'что случилось джон',
 'вчера вы опоздали сегодня вы опоздали',
 'нельзя опаздывать на урок',
 'вечером джон пришел домой',
 'дома он долго думал вчера я опоздал сегодня я тоже опоздал',
 'это очень плохо',
 'не буду больше опаздывать на урок',
 'завтра я встану рано в 8 часов быстро сделаю зарядку приготовлю завтрак позавтракаю повторю домашнее задание',
 'завтра я не опоздаю',
 'утром джон встал в 8 часов',
 'он сделал зарядку приготовил завтрак позавтракал повторил домашнее задание и пошел в институт',
 'в 9 часов 30 минут джон пришел в институт',
 'что случилось молодой человек',
 'куда вы идете',
 'спросил дежурный',
 'я иду на урок',
 'сегодня я не опоздал ответил джон',
 'идите домой молодой человек',
 'сегодня нет урока',
 'сегодня воскресенье']

In [9]:
def get_sent_gramm_features_map(clean_text):
    text_grammar_map = []
    m = Mystem()
    
    bar = progressbar.ProgressBar(maxval=len(clean_text),
                                      widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    line_number = 0
    bar.start()
        
    for sentence in clean_text:
        parsed_sentence_clean = []
        parsed_sentence =  m.analyze(sentence)
        for word in parsed_sentence:
            if word['text'].isalpha() or word['text'].isdigit():
                parsed_sentence_clean.append(word)
        text_grammar_map.append(parsed_sentence_clean)
        
        
        line_number += 1
        bar.update(line_number)
        time.sleep(0.1)
        
    return text_grammar_map
text_grammar_map_ex =  get_sent_gramm_features_map(sentences_list)

In [10]:
def lemmatize_text_from_grammar_map(text_gr_map):
    lemm_text = []
    for sentence_gr_map in text_gr_map:
        sentence = ''
        for word_grammar_features in sentence_gr_map:
            keys = list(word_grammar_features.keys())
            values = list(word_grammar_features.values())
            word  = word_grammar_features['text']
            #print(word)
            if(word.isdigit()):
                #print("digir")
                lemma = word
                sentence += lemma + ' '
            elif ('analysis' not in keys):
                pass
            elif(word_grammar_features['analysis'] ==[]):
                pass
            else:
                lemma = word_grammar_features['analysis'][0]['lex']
                sentence += lemma + ' '
        sentence = sentence.strip()
        lemm_text.append(sentence)
    return lemm_text
lemm_sentences = lemmatize_text_from_grammar_map(text_grammar_map_ex)

In [11]:
def get_weights_empty_list(clean_lemm_sentences_list, clean_orig_sentences_list):
    weights_list = []
    assert (len(clean_lemm_sentences_list) == len(clean_orig_sentences_list))
    
    for lemm_sentence, sentence in zip(clean_lemm_sentences_list,clean_orig_sentences_list):
        #print(lemm_sentence)
        #print(sentence)
        sentence_weights = []
        assert (len(lemm_sentence.split()) == len(sentence.split()))
        for lemm, word in zip (lemm_sentence.split(),sentence.split()) :
            
            if(lemm in russian_stopwords):
                weight = {"lemma" : lemm, "orig_word": word, "weight": 0, "stop_word":True}
            elif(lemm.isdigit()):
                weight = {"lemma" : lemm, "orig_word": word,"weight": 0, "digit":True}
            else:
                weight = {"lemma" : lemm, "orig_word": word,"weight": 0}
            sentence_weights.append(weight)
        weights_list.append(sentence_weights)
    return weights_list
text_weighs_1 = get_weights_empty_list(lemm_sentences, sentences_list)

In [39]:
with open("./adjusting texts/text_8_tokenized.parsed", "r", encoding = "utf-8") as f:
    word_count = 0
    for line in f.readlines():
        if(line[0].isdigit()):
            word_properties = line.split('\t')
            if(word_properties[3] != 'PUNCT'):
                print(word_properties)
                word_count += 1
    print(word_count)

['1', 'В', 'В', 'ADP', 'IN', '_', '2', 'case', '_', '_\n']
['2', 'классе', 'КЛАСС', 'NOUN', 'NN', 'Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing', '3', 'obl', '_', '_\n']
['3', 'был', 'БЫТЬ', 'VERB', 'VBC', 'Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin', '0', 'root', '_', '_\n']
['4', 'урок', 'урок', 'NOUN', 'NN', 'Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing', '3', 'nsubj', '_', '_\n']
['5', 'русского', 'РУССКИЙ', 'ADJ', 'JJL', 'Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing', '6', 'amod', '_', '_\n']
['6', 'языка', 'ЯЗЫК', 'NOUN', 'NN', 'Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing', '4', 'nmod', '_', 'SpaceAfter=No\n']
['1', 'Преподаватель', 'ПРЕПОДАВАТЕЛЬ', 'NOUN', 'NN', 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing', '2', 'nsubj', '_', '_\n']
['2', 'читал', 'ЧИТАТЬ', 'VERB', 'VBC', 'Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin', '0', 'root', '_', '_\n']
['3', 'новый', 'НОВЫЙ', 'ADJ', 'JJL', 'Animacy=Inan|Case=Acc|Gender=Masc|Number=

In [38]:
text_weighs_1

[[{'lemma': 'в', 'orig_word': 'в', 'stop_word': True, 'weight': 0},
  {'lemma': 'класс', 'orig_word': 'классе', 'weight': 0},
  {'lemma': 'быть', 'orig_word': 'был', 'stop_word': True, 'weight': 0},
  {'lemma': 'урок', 'orig_word': 'урок', 'weight': 0},
  {'lemma': 'русский', 'orig_word': 'русского', 'weight': 0},
  {'lemma': 'язык', 'orig_word': 'языка', 'weight': 0}],
 [{'lemma': 'преподаватель', 'orig_word': 'преподаватель', 'weight': 0},
  {'lemma': 'читать', 'orig_word': 'читал', 'weight': 0},
  {'lemma': 'новый', 'orig_word': 'новый', 'weight': 0},
  {'lemma': 'текст', 'orig_word': 'текст', 'weight': 0}],
 [{'lemma': 'студент', 'orig_word': 'студенты', 'weight': 0},
  {'lemma': 'внимательно', 'orig_word': 'внимательно', 'weight': 0},
  {'lemma': 'слушать', 'orig_word': 'слушали', 'weight': 0}],
 [{'lemma': 'извинять', 'orig_word': 'извините', 'weight': 0},
  {'lemma': 'можно', 'orig_word': 'можно', 'stop_word': True, 'weight': 0},
  {'lemma': 'входить', 'orig_word': 'войти', 'wei

In [12]:
def apply_udpipe_properties(ud_parsed_file, weights_input):
    weights_sentences = copy.deepcopy(weights_input)
    with open(ud_parsed_file, "r", encoding = "utf-8") as f:
        ud_properties = []
        for line in f.readlines():
            if(line[0].isdigit()):
                word_properties = line.split('\t')
                if(word_properties[3] != 'PUNCT'):
                    ud_properties.append(word_properties)
    nltk_word_count = 0 
    for sent_w in weights_input:
        nltk_word_count += len(sent_w)
    assert (nltk_word_count == len(ud_properties))
    ud_word_index = 0
    for sent_w in weights_sentences:
        for word_w in sent_w:
            #print(word_w['orig_word'])
            #print(ud_properties[ud_word_index][6])
            
            assert (word_w['orig_word'] == ud_properties[ud_word_index][1].lower())
            #print(type(word_w))
            word_w['synt_tree'] =OrderedDict()
            word_w['synt_tree']["word_index"] = ud_properties[ud_word_index][0]
            word_w['synt_tree']["head_word_index"] = ud_properties[ud_word_index][6]
            word_w['synt_tree']["univ_relat"] = ud_properties[ud_word_index][7]
            ud_word_index += 1
    return weights_sentences
            
text_weighs_synt = apply_udpipe_properties("./adjusting texts/text_8_tokenized.parsed", text_weighs_1)               

In [32]:
word_count = 0 
word_index
for sent_w in text_weighs_1:
    word_count += len(sent_w)
print(word_count)

133


In [13]:
def get_tf_idf_dict(lemm_text_list, save_to_csv = False):
    vect = TfidfVectorizer(stop_words = russian_stopwords)
    tfidf_matrix = vect.fit_transform(lemm_text_list)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    #print(df.head())
    if (save_to_csv): df.to_csv("./text_0_tfidf.xlsx", sep = '\t')
    tf_idf_dict = df.to_dict()
    return tf_idf_dict
tfidf_dict = get_tf_idf_dict(lemm_sentences, save_to_csv = True)

In [14]:
def assign_tf_idf(weights_list_input, tf_idf_dict):
    weights_list = copy.deepcopy(weights_list_input)
    for sentence_ind in range(len(weights_list)):
        for el_ind in range(len(weights_list[sentence_ind])):
            lemma = weights_list[sentence_ind][el_ind]["lemma"]
            
            if (lemma in tf_idf_dict):
                weights_list[sentence_ind][el_ind]["weight"] = tf_idf_dict[lemma][sentence_ind]
                #print(lemma, tf_idf_dict[lemma][sentence_ind])
            #else:
                #weights_list[sentence_ind][el_ind]["weight"] = 0.05
                #print(lemma, "not found")
                
    return weights_list
text_weighs_tfidf_2 = assign_tf_idf(text_weighs_synt, tfidf_dict)

In [15]:
text_weighs_tfidf_2

[[{'lemma': 'в',
   'orig_word': 'в',
   'stop_word': True,
   'synt_tree': OrderedDict([('word_index', '1'),
                ('head_word_index', '2'),
                ('univ_relat', 'case')]),
   'weight': 0},
  {'lemma': 'класс',
   'orig_word': 'классе',
   'synt_tree': OrderedDict([('word_index', '2'),
                ('head_word_index', '3'),
                ('univ_relat', 'obl')]),
   'weight': 0.5361619670142973},
  {'lemma': 'быть',
   'orig_word': 'был',
   'stop_word': True,
   'synt_tree': OrderedDict([('word_index', '3'),
                ('head_word_index', '0'),
                ('univ_relat', 'root')]),
   'weight': 0},
  {'lemma': 'урок',
   'orig_word': 'урок',
   'synt_tree': OrderedDict([('word_index', '4'),
                ('head_word_index', '3'),
                ('univ_relat', 'nsubj')]),
   'weight': 0.37093265612787246},
  {'lemma': 'русский',
   'orig_word': 'русского',
   'synt_tree': OrderedDict([('word_index', '5'),
                ('head_word_index', '6'),
  

# ИСКЛЮЧАЕМ ИЗ ПАЙПЛАЙН КЛЮЧЕВЫЕ СЛОВА

# конец исключения

In [16]:
def update_pos(weights_list, gr_map):
    
    sentences_weights_list_output = copy.deepcopy(weights_list)
    assert len(sentences_weights_list_output) == len(gr_map)#кол-во предложений
    for weights_sentence, gr_map_sentence in zip(sentences_weights_list_output, gr_map):
        assert len(weights_sentence) == len(gr_map_sentence )#кол-во слов
        for weight_word, gramm_map_word in zip(weights_sentence, gr_map_sentence):
            keys = list(gramm_map_word.keys())
            values = list(gramm_map_word.values())
            word  = gramm_map_word['text']
            #print(weight_word['word'])
            if(word.isdigit()):
                weight_word['POS'] = "number"
            elif ('analysis' not in keys):
                pass
            elif(gramm_map_word['analysis'] ==[]):
                pass
            else:
                grammar = gramm_map_word['analysis'][0]['gr']
                assert (weight_word['lemma'] == gramm_map_word['analysis'][0]['lex'])
                grammar_sep_by_comma = grammar.split(',')
                pos = re.match('[A-Z\s]+', grammar_sep_by_comma[0])[0]
                if(pos):
                    weight_word['POS'] = pos
                else:
                    weight_word['POS'] = "undefined"
                    
    return sentences_weights_list_output
pos_words_weights_4 = update_pos(text_weighs_tfidf_2, text_grammar_map_ex)

In [17]:
pos_words_weights_4

[[{'POS': 'PR',
   'lemma': 'в',
   'orig_word': 'в',
   'stop_word': True,
   'synt_tree': OrderedDict([('word_index', '1'),
                ('head_word_index', '2'),
                ('univ_relat', 'case')]),
   'weight': 0},
  {'POS': 'S',
   'lemma': 'класс',
   'orig_word': 'классе',
   'synt_tree': OrderedDict([('word_index', '2'),
                ('head_word_index', '3'),
                ('univ_relat', 'obl')]),
   'weight': 0.5361619670142973},
  {'POS': 'V',
   'lemma': 'быть',
   'orig_word': 'был',
   'stop_word': True,
   'synt_tree': OrderedDict([('word_index', '3'),
                ('head_word_index', '0'),
                ('univ_relat', 'root')]),
   'weight': 0},
  {'POS': 'S',
   'lemma': 'урок',
   'orig_word': 'урок',
   'synt_tree': OrderedDict([('word_index', '4'),
                ('head_word_index', '3'),
                ('univ_relat', 'nsubj')]),
   'weight': 0.37093265612787246},
  {'POS': 'A',
   'lemma': 'русский',
   'orig_word': 'русского',
   'synt_tree': Or

In [18]:
def create_lex_vector(lexema, pos):
    #если слова нет в часотном словаре, значит оно скорее всего довольно редкое и признаем его относящ к категории 2
    #если слово не заимствовано то признаем сложным 1 
    sence_value, freq_value, eng_value, abstract= 0,2,1,0
    diff_analysis = {}
    
    if(lexema in sence_dict):
        #print(lexema, "found")
        sence_value = sence_dict[lexema]
        
        
    if(lexema in freq_dict):
        freq_value = freq_dict[lexema]
    else:
        diff_analysis["freq_value_note"] = "not in freq dict"
    
    if(lexema in eng_words):
        eng_value = 0
    abs_suff={'аж', 'есть', 'ие', 'изм', 'изна', 'ина', 'ость', 'ота', 'ствo', 'ция',
 'чина','щина','ёж','еж'}   
    if(pos == "S"):
        for suff in abs_suff:
            if (lexema.endswith(suff)):
                abstract = 1

    diff_analysis['raw_diff_values'] = [sence_value, freq_value, eng_value, abstract]
    diff_analysis_named = OrderedDict()
    diff_analysis_named['sence_value'] = sence_value
    diff_analysis_named['freq_value'] = freq_value
    diff_analysis_named['eng_value'] = eng_value
    diff_analysis_named['abstract'] = abstract
    #diff_analysis_named = { "freq_value":freq_value, "eng_value":eng_value, "abstract":abstract, "sence_value":sence_value}

    
    return diff_analysis, diff_analysis_named

In [19]:
def update_with_lex_vector(weights_list_input):
    weights_list = copy.deepcopy(weights_list_input)
    for sentence_ind in range(len(weights_list)):
        for el_ind in range(len(weights_list[sentence_ind])):
            lemma = weights_list[sentence_ind][el_ind]["lemma"]
            if('digit' in weights_list[sentence_ind][el_ind]):
                diff_analysis['raw_diff_values'] = [0,0,0,0]
                diff_analysis_named = {"sence_value":0, "freq_value":0, "eng_value":0, "abstract":0}
            else:
                diff_analysis,diff_analysis_named  = create_lex_vector(lemma,weights_list[sentence_ind][el_ind]["POS"])
            weights_list[sentence_ind][el_ind]["lex_vector"] = {"raw": diff_analysis, "named": diff_analysis_named}
            
    return weights_list
lex_vector_weights_5 = update_with_lex_vector(pos_words_weights_4)

In [20]:
lex_vector_weights_5

[[{'POS': 'PR',
   'lemma': 'в',
   'lex_vector': {'named': OrderedDict([('sence_value', 2),
                 ('freq_value', 0),
                 ('eng_value', 1),
                 ('abstract', 0)]),
    'raw': {'raw_diff_values': [2, 0, 1, 0]}},
   'orig_word': 'в',
   'stop_word': True,
   'synt_tree': OrderedDict([('word_index', '1'),
                ('head_word_index', '2'),
                ('univ_relat', 'case')]),
   'weight': 0},
  {'POS': 'S',
   'lemma': 'класс',
   'lex_vector': {'named': OrderedDict([('sence_value', 1),
                 ('freq_value', 0),
                 ('eng_value', 1),
                 ('abstract', 0)]),
    'raw': {'raw_diff_values': [1, 0, 1, 0]}},
   'orig_word': 'классе',
   'synt_tree': OrderedDict([('word_index', '2'),
                ('head_word_index', '3'),
                ('univ_relat', 'obl')]),
   'weight': 0.5361619670142973},
  {'POS': 'V',
   'lemma': 'быть',
   'lex_vector': {'named': OrderedDict([('sence_value', 2),
                 ('fr

In [23]:
def update_coreference_negation(weight_list):
    weight_list_output = copy.deepcopy(weight_list)
    coreference_list = ["который","которая","это","этот","эти","которое"]
    negation_list = ["не","ни","бы"]
    all_sentences_map = []
    for sentence in weight_list_output:
        sentence_map = {'words':[],"negation_items":[],"coreference_items":[]}
        for word_weight in sentence:
            #print(word_weight)
            if word_weight['lemma'] in negation_list:
                sentence_map['negation_items'].append(word_weight['lemma'])
                word_weight['negation'] = True
            elif word_weight['lemma'] in coreference_list:
                sentence_map['coreference_items'].append(word_weight['lemma'])
                word_weight['coreference'] = True
            sentence_map['words'].append(word_weight)
        all_sentences_map.append(sentence_map)
    return all_sentences_map

sts_map = update_coreference_negation(lex_vector_weights_5)

In [24]:
sts_map

[{'coreference_items': [],
  'negation_items': [],
  'words': [{'POS': 'PR',
    'lemma': 'в',
    'lex_vector': {'named': OrderedDict([('sence_value', 2),
                  ('freq_value', 0),
                  ('eng_value', 1),
                  ('abstract', 0)]),
     'raw': {'raw_diff_values': [2, 0, 1, 0]}},
    'orig_word': 'в',
    'stop_word': True,
    'synt_tree': OrderedDict([('word_index', '1'),
                 ('head_word_index', '2'),
                 ('univ_relat', 'case')]),
    'weight': 0},
   {'POS': 'S',
    'lemma': 'класс',
    'lex_vector': {'named': OrderedDict([('sence_value', 1),
                  ('freq_value', 0),
                  ('eng_value', 1),
                  ('abstract', 0)]),
     'raw': {'raw_diff_values': [1, 0, 1, 0]}},
    'orig_word': 'классе',
    'synt_tree': OrderedDict([('word_index', '2'),
                 ('head_word_index', '3'),
                 ('univ_relat', 'obl')]),
    'weight': 0.5361619670142973},
   {'POS': 'V',
    'lemma': 'б

# переделать для sentence map

In [34]:
sts_map[0]['words'][1]

{'lemma': 'класс',
 'orig_word': 'классе',
 'weight': 0.5361619670142973,
 'synt_tree': OrderedDict([('word_index', '2'),
              ('head_word_index', '3'),
              ('univ_relat', 'obl')]),
 'POS': 'S',
 'lex_vector': {'raw': {'raw_diff_values': [1, 0, 1, 0]},
  'named': OrderedDict([('sence_value', 1),
               ('freq_value', 0),
               ('eng_value', 1),
               ('abstract', 0)])}}

In [51]:
#ПРОБЛЕМА --- В ИНДЕКСАХ УЧИТЫВАЮТСЯ ЗАПЯТЫЕ, КОТОРЫЕ НАМ НЕ НУЖНЫЮ РЕШИТЬ ПРОБЛЕМУ ЛДИСТОМ ОБОЙТИ ГЛАГОЛЫ
#НОВЫЕ ФИЧИ -  + КОЛ-ВО СЛОВ В ПРЯМОЙ ЗАВИСИМОСТИ ОТ РАССМАТРИВАЕМОГО + РАССТОЯНИЕ ОТ ГЛАВНОГО СЛОВА
def extract_dependencies (sentences_weights_input, debug = False):
    weights = copy.deepcopy(sentences_weights_input)
    """посчиать расстояние до главного слова
    посчитать кол-во зависимых слов"""
    
    for sentence_w in weights:
        #print(sentence_w)
        dep_dict = {}
        #ищем количество зависмых слов jn rf;ljuj ckjdf
        real_index = 1
        for word_w in sentence_w['words']:
            word_w['synt_tree']['real_index'] = real_index
            head_index = word_w['synt_tree']['head_word_index']
            #print(word_w)
            #print(head_index)
            if(head_index in dep_dict):
                #print("kek")
                dep_dict[head_index] += 1
            elif(head_index not in dep_dict and head_index!=0):
                #print("bek")
                dep_dict[head_index] = 1
            #print(dep_dict)
            real_index += 1
            
        """присваиваем посчитаное количество зависимых слов прибалвяя 1"""
        for word_w in sentence_w['words']:
            if(word_w['synt_tree']['word_index'] in dep_dict):
                #print("current_word index", word_w['synt_tree']['word_index'])
                #print("dependent words count", dep_dict[word_w['synt_tree']['word_index']])
                word_w['synt_tree']['dep_words_count'] = dep_dict[word_w['synt_tree']['word_index']] + 1 
            else:
                word_w['synt_tree']['dep_words_count'] = 1
                
        dependencies_dist = []
        for word_w in sentence_w['words']:
            if debug: print("DITANCE BETWEEN",word_w)
            current_word_real_index = word_w['synt_tree']['real_index']
            nominal_head_index = int(word_w['synt_tree']['head_word_index'])
            if(nominal_head_index != 0):
                #look for head word
                for check_index_word_w in sentence_w['words']:
                    if (check_index_word_w != word_w):
                        nominal_word_index = int(check_index_word_w['synt_tree']['word_index'])
                        if (nominal_word_index == nominal_head_index):
                            if debug:print("AND",check_index_word_w)
                            real_head_index = check_index_word_w['synt_tree']['real_index']
                            distance = abs(real_head_index - current_word_real_index)
                            if debug:print("==", distance)
                            dependencies_dist.append(distance)
                            break
        if debug:
            print(sentence_w['words'])
            #print(dependencies_dist)
        sentence_w['dependencies_length'] = dependencies_dist
    return weights
sts_map_dep = extract_dependencies(sts_map, debug = False)     

In [52]:
sts_map_dep

[{'coreference_items': [],
  'dependencies_length': [1, 1, 1, 1, 2],
  'negation_items': [],
  'words': [{'POS': 'PR',
    'lemma': 'в',
    'lex_vector': {'named': OrderedDict([('sence_value', 2),
                  ('freq_value', 0),
                  ('eng_value', 1),
                  ('abstract', 0)]),
     'raw': {'raw_diff_values': [2, 0, 1, 0]}},
    'orig_word': 'в',
    'stop_word': True,
    'synt_tree': OrderedDict([('word_index', '1'),
                 ('head_word_index', '2'),
                 ('univ_relat', 'case'),
                 ('real_index', 1),
                 ('dep_words_count', 1)]),
    'weight': 0},
   {'POS': 'S',
    'lemma': 'класс',
    'lex_vector': {'named': OrderedDict([('sence_value', 1),
                  ('freq_value', 0),
                  ('eng_value', 1),
                  ('abstract', 0)]),
     'raw': {'raw_diff_values': [1, 0, 1, 0]}},
    'orig_word': 'классе',
    'synt_tree': OrderedDict([('word_index', '2'),
                 ('head_word_ind

In [53]:
def update_weight(s_weight, w_weight, pos):
    if ("special_pos" in w_weight):
        w_weight['special_pos'].append(pos)
    else:
        w_weight['special_pos'] = []
        w_weight['special_pos'].append(pos)
    s_weight['sent_special_pos'].append(w_weight['lemma'] + '_' + pos)

def update_special_pos(sentences_weights_list, gr_map):
    special_pos_list = ["инф", "прич", "деепр"]
    
    sentences_weights_list_output = copy.deepcopy(sentences_weights_list)
    assert len(sentences_weights_list_output) == len(gr_map)#кол-во предложений
    for weights_sentence, gr_map_sentence in zip(sentences_weights_list_output, gr_map):
        assert len(weights_sentence['words']) == len(gr_map_sentence )#кол-во слов
        weights_sentence['sent_special_pos'] = []
        for weight_word, gramm_map_word in zip(weights_sentence['words'], gr_map_sentence):
            keys = list(gramm_map_word.keys())
            values = list(gramm_map_word.values())
            word  = gramm_map_word['text']
            #print(weight_word['word'])
            if ('analysis' not in keys):
                pass
            elif(gramm_map_word['analysis'] ==[]):
                pass
            else:
                grammar = gramm_map_word['analysis'][0]['gr']
                assert (weight_word['lemma'] == gramm_map_word['analysis'][0]['lex'])
                for spec_pos in special_pos_list:
                    if (spec_pos in grammar):
                        update_weight(weights_sentence, weight_word,spec_pos)
                grammar_sep_by_comma = grammar.split(',')
                if (len(grammar_sep_by_comma) == 1):
                    pass
                else:
                    pos = re.match('[A-Z\s]+', grammar_sep_by_comma[0])[0]
                    if(pos == "V"):
                        if(gramm_map_word["text"].endswith("сь") or gramm_map_word["text"].endswith("ся")):
                            update_weight(weights_sentence, weight_word,"возвратный")
    return sentences_weights_list_output
sts_map_spec_2 = update_special_pos(sts_map_dep, text_grammar_map_ex)

In [55]:
def calculate_lix_from_list_of_sentences(processed_text_sentences):
        sentences_count = len(processed_text_sentences)
        words_count = sum([len(line.split(' ')) for line in processed_text_sentences])
        long_words_count = 0 #more than 6
        for line in processed_text_sentences:
            for word in line.split():
                if len(word) > 6:
                    long_words_count += 1
        lix = words_count/ sentences_count + (long_words_count * 100) / words_count
        
        return round(lix,2)
text_lix = calculate_lix_from_list_of_sentences(sentences_list)

In [56]:
def calculate_type_token_ratio(lemm_text_sentences):
      all_words = []
      for sentence in lemm_text_sentences:
          words = sentence.split()
          for word in words:
              all_words.append(word)

      unqie_words = set(all_words)
      types = len(unqie_words)
      tokens = len (all_words)

      return round(types/tokens,2)

In [60]:
def caclulate_overall_text_features(sentences_map, processed_text_sentences, lemm_text_sentences, raw_aligned_text):
    text_map = {}
    non_stop_word_count = 0
    verbs_count = 0
    inf, prich, deeprich, vozvr = 0,0,0,0
    all_spec_pos = 0
    negation_count, coreference_count = 0,0
    
    
    for sentence in sentences_map:
        for spec_pos in sentence['sent_special_pos']:
            if("_инф" in spec_pos):
                
                inf += 1
                all_spec_pos += 1
            elif("_прич" in spec_pos):
                
                prich += 1
                all_spec_pos += 1
            elif("_деепр" in spec_pos):
                
                deeprich += 1
                all_spec_pos += 1
            elif("_возвратный" in spec_pos):
                
                vozvr += 1
        negation_count += len(sentence['negation_items'])      
        coreference_count += len(sentence['coreference_items']) 
        
        for word in sentence['words']:
            #print(word)
            if (word["POS"] == "V"):
                verbs_count += 1
            if("stop_word" not in word):
                non_stop_word_count += 1
                
        
    text_map = OrderedDict()
    text_map['lix'] = calculate_lix_from_list_of_sentences(processed_text_sentences)
    text_map['ttr'] = calculate_type_token_ratio(lemm_text_sentences)
    #text_map['inf_verb'] = inf/verbs_count
    #text_map['prich_verb'] = prich/verbs_count
    #text_map['deeprich_verb'] = deeprich/verbs_count
    #text_map['vozvr_verb'] = vozvr/verbs_count
    text_map['spec_pos_verb'] = round(all_spec_pos/ verbs_count,3)
    
    text_map['negation_per_n_stop_words'] =  round(negation_count/non_stop_word_count,3)
    text_map ['coreference_per_n_stop_words'] =  round(coreference_count/non_stop_word_count,3)
    
    text_map['sentences&words_analysis'] = sentences_map
    
    text_map['raw_aligned_text'] = raw_aligned_text
    text_map['cleaned_sentences'] = processed_text_sentences
    text_map['lemm_sentences'] = lemm_text_sentences
    
    return  text_map
text_map = caclulate_overall_text_features(sts_map_spec_2, sentences_list,lemm_sentences, list_sentences_lines)

In [62]:
def calculate_text_map(file,udpipe_file):
    raw_aligned_sentences = delete_empty_newlines(file)
    clean_sentences_list = clean_file(raw_aligned_sentences)
    text_grammar_map =  get_sent_gramm_features_map(clean_sentences_list)
    lemm_sentences_list = lemmatize_text_from_grammar_map(text_grammar_map)
    empty_weights_1 = get_weights_empty_list(lemm_sentences_list, clean_sentences_list)
    
    text_weighs_synt = apply_udpipe_properties(udpipe_file, empty_weights_1) 
    
    tfidf_dict = get_tf_idf_dict(lemm_sentences_list)
    text_weighs_tfidf_2 = assign_tf_idf(text_weighs_synt, tfidf_dict)
    pos_words_weights_4 = update_pos(text_weighs_tfidf_2, text_grammar_map)
    lex_vector_weights_5 = update_with_lex_vector(pos_words_weights_4)
    sentence_map_1 = update_coreference_negation(lex_vector_weights_5)
    sts_map_dep = extract_dependencies(sentence_map_1, debug = False)   
    sentence_spec_2 = update_special_pos(sts_map_dep, text_grammar_map)
    text_map = caclulate_overall_text_features(sentence_spec_2, clean_sentences_list,lemm_sentences_list, raw_aligned_sentences)
    return text_map
calculate_text_map("text_8.txt","./adjusting texts/text_8_tokenized.parsed")  

OrderedDict([('lix', 40.66),
             ('ttr', 0.53),
             ('spec_pos_verb', 0.081),
             ('negation_per_n_stop_words', 0.03),
             ('coreference_per_n_stop_words', 0.01),
             ('sentences&words_analysis',
              [{'coreference_items': [],
                'dependencies_length': [1, 1, 1, 1, 2],
                'negation_items': [],
                'sent_special_pos': [],
                'words': [{'POS': 'PR',
                  'lemma': 'в',
                  'lex_vector': {'named': OrderedDict([('sence_value', 2),
                                ('freq_value', 0),
                                ('eng_value', 1),
                                ('abstract', 0)]),
                   'raw': {'raw_diff_values': [2, 0, 1, 0]}},
                  'orig_word': 'в',
                  'stop_word': True,
                  'synt_tree': OrderedDict([('word_index', '1'),
                               ('head_word_index', '2'),
                            